In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Brandeis/train.zip /content
%cd /content
!unzip -o train.zip

cp: cannot stat '/content/drive/MyDrive/Brandeis/train.zip': No such file or directory
/content
unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [ ]:
!pip install git+https://github.com/ufoym/imbalanced-dataset-sampler.git

In [ ]:
import shutil,os
os.chdir('/content/drive/MyDrive/Brandeis/COSI-149B/')
import numpy as np
import pandas as pd
from PIL import Image
from skimage import io
import torch
print(torch.__version__)
import torch.nn as NN
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
from torchvision import models
import argparse
import time
from tqdm import tqdm_notebook as tqdm
from torchsampler import ImbalancedDatasetSampler

1.8.1+cu101


In [ ]:
class CSVDataset(Dataset):
  def __init__(self, df, path, transform0=None, transform1=None, balance=True):
      # loads csv file as a dataframe object
      self.df = df
      self.transform0 = transform0
      self.transform1 = transform1
      self.path = path
      if balance:
        df_pos = df[df['class'] > 0]
        df_neg = df[df['class'] == 0]
        len_pos = len(df_pos)
        len_neg = len(df_neg)
        df_pos_new = df_pos.sample(n = len_neg, axis = 0, replace = True)
        self.df = pd.concat([df_pos_new, df_neg], axis = 0)
        self.df = self.df.sample(frac=1).reset_index(drop=True)

  def __len__(self):
      return len(self.df)

  def __getitem__(self, index):
      # load label
      lbl = self.df.iloc[index]
      img_path = lbl['filename']
      # load class
      if lbl['class'] != 0:
        img_class = 1
      else:
        img_class = 0
      # convert path to load images on HPCC
      img_path = self.path + img_path
      img = Image.open(img_path).convert('RGB')
      if img_class == 0:
        img = self.transform0(img)
      else: 
        img = self.transform1(img)
      return img, img_class

In [ ]:
df=pd.read_csv("data.csv")

In [ ]:
use_gpu = torch.cuda.is_available()
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Apr 12 02:09:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def callback(dataset, idx):
  return dataset[idx][1]

df_all = df.copy()
df_n = pd.read_csv("n.csv")
df_p = pd.read_csv("p.csv")

mean = (0.7192409, 0.7192409, 0.7192409)
std = (0.01884892, 0.01884892, 0.01884892)

testTransform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

transform_set = [
    transforms.CenterCrop((int(129*0.8))),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(degrees = 90)
]
trainTransform = transforms.Compose([
    transforms.RandomChoice(transform_set),
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


train_batchsize = 64
test_batchsize = 10
trainEpochsInit = 100
trainEpochs = 50
learningRate = 0.0001
sgdmomentum = 0.9
betas = (0.9, 0.999)
eps = 1e-08
weight_decay=0
batch_init = 1
partition_init = 1


In [ ]:
def genDataLoader(batch, partition):
  div1_n = int(len(df_n)/10*(10-batch))
  div2_n = int(len(df_n)/10*(10-batch+1))
  df_n_train = df_n[0:div1_n].copy().append(df_n[div2_n:])
  df_n_test = df_n[div1_n: div2_n].copy()

  div1_p = int(len(df_p)/10*(10-batch))
  div2_p = int(len(df_p)/10*(10-batch+1))
  df_p_train = df_p[0:div1_p].copy().append(df_p[div2_p:])
  df_p_test = df_p[div1_p: div2_p].copy()


  if partition == 1:
    df_train_all = df_n_train[0:int(len(df_n_train)/2)].append(df_p_train)
  else:
    df_train_all = df_n_train[int(len(df_n_train)/2):].append(df_p_train)
  df_train_all = df_train_all.sample(frac=1).reset_index(drop=True)
  df_test_all = df_n_test.append(df_p_test)
  df_test_all = df_test_all.sample(frac=1).reset_index(drop=True)

  trainSet = CSVDataset(df_train_all, '/content/', trainTransform, trainTransform)
  testSet = CSVDataset(df_test_all, '/content/', testTransform, testTransform, False)
  # sampler = WeightedRandomSampler(weights, len(weights), True)
  # trainLoader = DataLoader(trainSet, batch_size=train_batchsize, sampler=ImbalancedDatasetSampler(trainSet, callback_get_label=callback))
  trainLoader = DataLoader(trainSet, batch_size=train_batchsize, shuffle=False)
  testLoader = DataLoader(testSet, batch_size=test_batchsize, shuffle=False)
  return trainLoader, testLoader


In [ ]:
from torch.autograd import Variable
class FocalLoss(NN.Module):
    r"""
        This criterion is a implemenation of Focal Loss, which is proposed in 
        Focal Loss for Dense Object Detection.

            Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])

        The losses are averaged across observations for each minibatch.

        Args:
            alpha(1D Tensor, Variable) : the scalar factor for this criterion
            gamma(float, double) : gamma > 0; reduces the relative loss for well-classiﬁed examples (p > .5), 
                                   putting more focus on hard, misclassiﬁed examples
            size_average(bool): By default, the losses are averaged over observations for each minibatch.
                                However, if the field size_average is set to False, the losses are
                                instead summed for each minibatch.


    """
    def __init__(self, class_num, alpha=None, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        if alpha is None:
            self.alpha = Variable(torch.ones(class_num, 1))
        else:
            if isinstance(alpha, Variable):
                self.alpha = alpha
            else:
                self.alpha = Variable(alpha)
        self.gamma = gamma
        self.class_num = class_num
        self.size_average = size_average

    def forward(self, inputs, targets):
        N = inputs.size(0)
        C = inputs.size(1)
        P = F.softmax(inputs, dim=1)

        class_mask = inputs.data.new(N, C).fill_(0)
        class_mask = Variable(class_mask)
        ids = targets.view(-1, 1)
        class_mask.scatter_(1, ids.data, 1.)
        #print(class_mask)


        if inputs.is_cuda and not self.alpha.is_cuda:
            self.alpha = self.alpha.cuda()
        alpha = self.alpha[ids.data.view(-1)]

        probs = (P*class_mask).sum(1).view(-1,1)

        log_p = probs.log()

        batch_loss = -alpha*(torch.pow((1-probs), self.gamma))*log_p 

        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss

In [ ]:
# Define the model
class Net(NN.Module):
    def __init__(self, num_classes=2):
      super(Net, self).__init__()
      net = models.vgg16_bn(pretrained=False) 
      net.classifier = NN.Sequential()	
      self.features = net		
      self.classifier = NN.Sequential(  
              NN.Linear(512 * 7 * 7, 4096),
              NN.ReLU(True),
              NN.Dropout(p=0.5),
              NN.Linear(4096, 4096),
              NN.ReLU(True),
              NN.Dropout(p=0.5),
              NN.Linear(4096, num_classes),
      )

    def forward(self, x):
      x = self.features(x)
      x = x.view(x.size(0), -1)
      x = self.classifier(x)
      return F.log_softmax(x, dim=1)

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        criterion = FocalLoss(class_num = 2)
        if (use_gpu):
            model = model.cuda()
            data = data.cuda()
            target = target.cuda()
            criterion = criterion.cuda()
        optimizer.zero_grad()
        output = model( data )
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    totalT = 0
    totalF = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    with torch.no_grad():
        for data, target in test_loader:
            criterion = FocalLoss(class_num = 2, size_average = False)
            if (use_gpu):
                model = model.cuda()
                data = data.cuda()
                target = target.cuda()
                criterion = criterion.cuda()
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            for i in range(len(target)):
              if target[i] == 1:
                totalT = totalT+1
                if target[i] == pred[i]:
                  TP = TP+1
                else:
                  FN = FN + 1
              else:
                totalF = totalF+1
                if target[i] == pred[i]:
                  TN = TN+1
                else:
                  FP = FP + 1

    test_loss /= len(test_loader.dataset)
    recall = 1.* TP / (TP + FN + 1e-8)
    precision = 1. * TP / (TP + FP + 1e-8)
    F1 = 2*recall*precision/(recall+precision+1e-8)

    print('\nTest set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print('False: {}/{} True: {}/{}\n'.format(
        TN, totalF, TP, totalT
    ))
    print('recall: {}\tprecision: {}\tF1: {}\n'.format(recall, precision, F1))
    return test_loss

In [ ]:
# Train and test
model = Net()
optimizer = optim.Adam(model.parameters(), lr=learningRate, betas=betas, eps=eps, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=10, verbose=True, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

epochs_train = trainEpochs
partition = partition_init
for batch in range(batch_init, 10):
  while (partition < 3):
    print(" ============= Batch " + str(batch) + " P " + str(partition) + " Start ==============\n")
    if batch_init == 1 & partition == 1:
      epochs_train = trainEpochsInit
    elif partition == 1:
      model.load_state_dict(torch.load("cnn_vgg16_train_" + str(batch-1) + "_2.pt"))
    else:
      model.load_state_dict(torch.load("cnn_vgg16_train_" + str(batch) + "_1.pt"))

    trainLoader, testLoader = genDataLoader(batch, partition)
    
    pre_loss = 0
    stoper = 0
    for epoch in range(epochs_train):
        train(model, trainLoader, optimizer, epoch)
        loss = test(model, testLoader)
        scheduler.step(loss)

        if epoch > 50:
          if loss > pre_loss:
            stoper = stoper + 1
          else:
            stoper = 0
          if stoper == 5:
            print("Stop at epoch " + str(epoch) + " >>>>>>>>>>>>>")
            stoper = 0
            break
          pre_loss = loss

          if epoch % 5 == 0:
            torch.save(model.state_dict(),"cnn_vgg16_train_" + str(batch) + "_" + str(partition) + ".pt")
        else:
          if epoch % 5 == 0:
            torch.save(model.state_dict(),"cnn_vgg16_train_" + str(batch) + "_" + str(partition) + ".pt")
    partition = partition + 1
  partition = 1

In [ ]:
# test the model on all train dataset
allSet = CSVDataset(df_all, '/content/', testTransform, testTransform, False)
allLoader = DataLoader(allSet, batch_size=test_batchsize, shuffle=False)
model = Net()
partition = 1
for batch in range(1, 10):
  while (partition < 3):
    print(" ============= Test Batch " + str(batch) + " P " + str(partition) + " Start ==============\n")
    model.load_state_dict(torch.load("cnn_vgg19_result/cnn_vgg16_train_" + str(batch) + "_" + str(partition) + ".pt"))
    test(model, allLoader)
    partition = partition + 1
  partition = 1